In [79]:
import numpy as np

from catboost import CatBoostClassifier

from sklearn.datasets import make_classification

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split

from sklearn.metrics import accuracy_score,classification_report

In [80]:
# Generate synthetic data

X,y = make_classification(n_samples=1000,n_features=20, n_classes=2, random_state=42)

In [81]:
# Split the data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [82]:
# Initialize base CatBoost model

base_model = CatBoostClassifier(verbose=0, random_state=42)

In [83]:
base_model.fit(X_train,y_train)

In [84]:
# Base Model Prediction

y_pred = base_model.predict(X_test)

In [85]:
# Base Model Accuracy

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy of Base Model : {accuracy}")


Accuracy of Base Model : 0.87


In [86]:
# Define parameter grid for RandomizedSearchCV

param_dist = {

    'iterations': [100,200,300],
    'learning_rate': np.linspace(0.01,0.3,5),
    'depth' : [4,6,8,10],
    'l2_leaf_reg':[1,3,5,7,9],
    'border_count': [32,64,128]
}

In [87]:
# Perform Randomized Search

random_search = RandomizedSearchCV(

    estimator=CatBoostClassifier(verbose=0,random_state=42),
    param_distributions=param_dist,
    n_iter=20,
    cv=3,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1

)

In [88]:
# Fit with random search

random_search.fit(X_train,y_train)

RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostClassifier object at 0x144313ac0>,
                   n_iter=20, n_jobs=-1,
                   param_distributions={'border_count': [32, 64, 128],
                                        'depth': [4, 6, 8, 10],
                                        'iterations': [100, 200, 300],
                                        'l2_leaf_reg': [1, 3, 5, 7, 9],
                                        'learning_rate': array([0.01  , 0.0825, 0.155 , 0.2275, 0.3   ])},
                   random_state=42, scoring='accuracy')

In [89]:
# Random Search Best Parameters

print(f"Random Search Best Parameter : {random_search.best_params_}")

Random Search Best Parameter : {'learning_rate': 0.22749999999999998, 'l2_leaf_reg': 5, 'iterations': 300, 'depth': 10, 'border_count': 64}


In [90]:
# Train the best model from Random Search

best_random_model = CatBoostClassifier(**random_search.best_params_,verbose=0,random_state=42)

In [91]:
# Fit Best Random model

best_random_model.fit(X_train,y_train)

In [92]:
# Predict with best random model

y_pred = best_random_model.predict(X_test)

In [93]:
# Best Random model Accuracy

print("Tuned Model Accuracy (Random Search):", accuracy_score(y_test, y_pred))


Tuned Model Accuracy (Random Search): 0.8666666666666667


In [94]:
# Define a smaller parameter grid for GridSearchCV

param_grid = {

    'iterations' : [random_search.best_params_['iterations']],
    'learning_rate' : [random_search.best_params_['learning_rate']],
    'depth' : [random_search.best_params_['depth']-1,random_search.best_params_['depth'],random_search.best_params_['depth'] +1],
    'l2_leaf_reg': [random_search.best_params_['l2_leaf_reg'] - 1, random_search.best_params_['l2_leaf_reg'], random_search.best_params_['l2_leaf_reg'] + 1]


}

In [95]:
# Perform Grid Search

grid_search = GridSearchCV(

    estimator=CatBoostClassifier(verbose=0, random_state=42),
    param_grid=param_grid,
    cv=3,
    scoring='accuracy',
    n_jobs=-1
)

In [96]:
# Fit the Grid search model

grid_search.fit(X_train,y_train)

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostClassifier object at 0x14431caf0>,
             n_jobs=-1,
             param_grid={'depth': [9, 10, 11], 'iterations': [300],
                         'l2_leaf_reg': [4, 5, 6],
                         'learning_rate': [0.22749999999999998]},
             scoring='accuracy')

In [97]:
# Grid Search Best Parameters

print("Best Parameters (Grid Search):", grid_search.best_params_)

Best Parameters (Grid Search): {'depth': 10, 'iterations': 300, 'l2_leaf_reg': 5, 'learning_rate': 0.22749999999999998}


In [99]:
# Train the best model from Grid Search

best_grid_model = CatBoostClassifier(**grid_search.best_params_,verbose=0, random_state=42)

In [100]:
# Fit with best grid model

best_grid_model.fit(X_train,y_train)

In [101]:
# Predict with best grid model

y_pred = best_grid_model.predict(X_test)

In [102]:
# Evaluate Best grid Model 

print("Tuned Model Accuracy (Grid Search):", accuracy_score(y_test, y_pred))

Tuned Model Accuracy (Grid Search): 0.8666666666666667


In [103]:
# Summary of results

print("\nClassification Report for Best Model:")
print(classification_report(y_test, y_pred))


Classification Report for Best Model:
              precision    recall  f1-score   support

           0       0.84      0.90      0.87       145
           1       0.90      0.84      0.87       155

    accuracy                           0.87       300
   macro avg       0.87      0.87      0.87       300
weighted avg       0.87      0.87      0.87       300

